In [2]:
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install tensorflow==1.15
!pip install bert-tensorflow==1.0.1

In [4]:
import os
import re
import pandas as pd
import numpy as np 
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [5]:
print(tf.__version__)
print (sys.version)

1.15.0
3.6.13 |Anaconda, Inc.| (default, Mar 16 2021, 11:37:27) [MSC v.1916 64 bit (AMD64)]


In [6]:
import warnings
import logging, sys

logging.disable(sys.maxsize)
warnings.filterwarnings("ignore")

In [7]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16996312281412840453
]


In [8]:
import bert

#Load submodules
from bert import run_classifier
from bert import optimization
from bert import tokenization

Import Datas

In [9]:
# from google.colab import files
# train_upload = files.upload()
# test_upload = files.upload()

In [9]:
import io
path = '../data/' 

train = pd.read_csv(path + 'dreaddit-train.csv', encoding = "ISO-8859-1")
test = pd.read_csv(path + 'dreaddit-test.csv', encoding = "ISO-8859-1")

In [10]:
train.head()

,subreddit,post_id,sentence_range,text,id,label,confidence,social_timestamp,social_karma,syntax_ari,...,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",33181,1,0.8,1521614353,5,1.806818,...,1.000,1.1250,1.0,1.77000,1.52211,1.89556,0.86,1,3.253573,-0.002742
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",2606,0,1.0,1527009817,4,9.429737,...,1.125,1.0000,1.0,1.69586,1.62045,1.88919,0.65,2,8.828316,0.292857
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,38816,1,0.8,1535935605,2,7.769821,...,1.000,1.1429,1.0,1.83088,1.58108,1.85828,0.67,0,7.841667,0.011894
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",239,1,0.6,1516429555,0,2.667798,...,1.000,1.1250,1.0,1.75356,1.52114,1.98848,0.50,5,4.104027,0.141671
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1421,1,0.8,1539809005,24,7.554238,...,1.000,1.1250,1.0,1.77644,1.64872,1.81456,1.00,1,7.910952,-0.204167


In [11]:
DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'
# label_list contains list of labels (i.e stress, non-stress)
label_list = [0, 1]

In [12]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [13]:
# Load a vocabulary file and lowercasing information directly from the BERT tf hub module

# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                                tokenization_info["do_lower_case"]])

    return bert.tokenization.FullTokenizer(vocab_file=vocab_file, 
                                    do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

In [14]:
# Set the maximum sequence length. 
def get_max_len(text):
    max_len = 0
    for i in range(len(train)):
        if len(text.iloc[i]) > max_len:
            max_len = len(text.iloc[i])
    return max_len

temp = train.text.str.split(' ')
max_len = get_max_len(temp)

In [15]:
MAX_SEQ_LENGTH = max_len
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, 
                                                                  label_list, 
                                                                  MAX_SEQ_LENGTH, 
                                                                  tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, 
                                                                 label_list, 
                                                                 MAX_SEQ_LENGTH, 
                                                                 tokenizer)

Classification Model

In [16]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
    """Creates a classification model."""

    bert_module = hub.Module(BERT_MODEL_HUB, trainable=True)
    bert_inputs = dict(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids)
    bert_outputs = bert_module(inputs=bert_inputs, signature="tokens", as_dict=True)

    # Use "pooled_output" for classification tasks on an entire sentence.
    # Use "sequence_outputs" for token-level output.
    output_layer = bert_outputs["pooled_output"]

    hidden_size = output_layer.shape[-1].value

    # Create our own layer to tune for politeness data.
    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable("output_bias", 
                                  [num_labels], 
                                  initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):

        # Dropout helps prevent overfitting
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        # unshaped_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        # predicted_labels = tf.reshape(unshaped_labels, [])
        predicted_labels = tf.argmax(log_probs, axis=-1, output_type=tf.int32)
        # If we're predicting, we want predicted labels and the probabiltiies.
        if is_predicting:
            return (predicted_labels, log_probs)

        # If we're train/eval, compute loss between predicted and actual label
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, predicted_labels, log_probs)

In [17]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps, num_warmup_steps):
    """Returns `model_fn` closure for TPUEstimator."""
    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""
        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
        # TRAIN and EVAL
        if not is_predicting:
            (loss, predicted_labels, log_probs) = create_model(is_predicting, 
                                                               input_ids, 
                                                               input_mask, 
                                                               segment_ids, 
                                                               label_ids, 
                                                               num_labels)

            train_op = bert.optimization.create_optimizer(loss, 
                                                          learning_rate, 
                                                          num_train_steps, 
                                                          num_warmup_steps, 
                                                          use_tpu=False)

            # Calculate evaluation metrics. 
            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                f1_score = tf.contrib.metrics.f1_score(label_ids, predicted_labels)
                auc = tf.metrics.auc(label_ids, predicted_labels)
                recall = tf.metrics.recall(label_ids, predicted_labels)
                precision = tf.metrics.precision(label_ids, predicted_labels) 
                true_pos = tf.metrics.true_positives(label_ids, predicted_labels)
                true_neg = tf.metrics.true_negatives(label_ids, predicted_labels)   
                false_pos = tf.metrics.false_positives(label_ids, predicted_labels)  
                false_neg = tf.metrics.false_negatives(label_ids, predicted_labels)

                return {
                    "eval_accuracy": accuracy,
                    "f1_score": f1_score,
                    "auc": auc,
                    "precision": precision,
                    "recall": recall,
                    "true_positives": true_pos,
                    "true_negatives": true_neg,
                    "false_positives": false_pos,
                    "false_negatives": false_neg
                }

            eval_metrics = metric_fn(label_ids, predicted_labels)

            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metrics)
        else:
            (predicted_labels, log_probs) = create_model(is_predicting, 
                                                         input_ids, 
                                                         input_mask, 
                                                         segment_ids, 
                                                         label_ids, 
                                                         num_labels)

            predictions = {'probabilities': log_probs, 'labels': predicted_labels}
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Return the actual model function in the closure
    return model_fn

In [18]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3
# Warmup is a period of time where the learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [19]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [20]:
# Specify outpit directory and number of checkpoint steps to save
OUTPUT_DIR = '../output'

run_config = tf.estimator.RunConfig(model_dir=OUTPUT_DIR,
                                    save_summary_steps=SAVE_SUMMARY_STEPS,
                                    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [21]:
model_fn = model_fn_builder(num_labels=len(label_list), 
                            learning_rate=LEARNING_RATE,
                            num_train_steps=num_train_steps,
                            num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(model_fn=model_fn,
                                   config=run_config,
                                   params={"batch_size": BATCH_SIZE})

In [22]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(features=train_features,
                                                      seq_length=MAX_SEQ_LENGTH,
                                                      is_training=True,
                                                      drop_remainder=False)

Fine-tuning BERT

In [23]:
print(f'Training Started')
current_time = datetime.now()

# train the model 
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

print("Training took ", datetime.now() - current_time)

Training Started
Training took  0:00:00.014658


In [24]:
# check the test result
test_input_fn = run_classifier.input_fn_builder(features=test_features,
                                                seq_length=MAX_SEQ_LENGTH,
                                                is_training=False,
                                                drop_remainder=False)

In [26]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

{'auc': 0.8033624,
 'eval_accuracy': 0.8041958,
 'f1_score': 0.8138297,
 'false_negatives': 63.0,
 'false_positives': 77.0,
 'loss': 0.83963794,
 'precision': 0.7989556,
 'recall': 0.8292683,
 'true_negatives': 269.0,
 'true_positives': 306.0,
 'global_step': 532}

In [25]:
def predict(in_sentences):
    CLASSES = ["non-stress", "stress"]

    input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label

    input_features = run_classifier.convert_examples_to_features(input_examples, 
                                                                 label_list, 
                                                                 MAX_SEQ_LENGTH, 
                                                                 tokenizer)
    
    predict_input_fn = run_classifier.input_fn_builder(features=input_features, 
                                                       seq_length=MAX_SEQ_LENGTH, 
                                                       is_training=False, 
                                                       drop_remainder=False)

    predictions = estimator.predict(predict_input_fn)
    return [{"input": sentence, "probabilities": list(prediction['probabilities']), "class": CLASSES[prediction['labels']]}
            for sentence, prediction in zip(in_sentences, predictions)]


In [26]:
pred_sentences = ["Wow it's such a nice day today! I want to eat ice cream and have fun.",
"Man. I'm so lazy today and just want to sleep all day. I don't feel like doing anything after that breakup.",
"Do you like fruits? I like them so much! I'm very sad and depressed."
]

In [27]:
predictions = predict(pred_sentences)
predictions

[{'input': "Wow it's such a nice day today! I want to eat ice cream and have fun.",
  'probabilities': [-0.0026399782, -5.9383264],
  'class': 'non-stress'},
 {'input': "Man. I'm so lazy today and just want to sleep all day. I don't feel like doing anything after that breakup.",
  'probabilities': [-4.1206646, -0.016366992],
  'class': 'stress'},
 {'input': "Do you like fruits? I like them so much! I'm very sad and depressed.",
  'probabilities': [-6.314957, -0.0018106985],
  'class': 'stress'}]

Save to File

In [29]:
import os
import tensorflow as tf

trained_checkpoint_prefix = '../output/model.ckpt-532'
export_dir = '../bert_output/'

graph = tf.Graph()
with tf.compat.v1.Session(graph=graph) as sess:
    # Restore from checkpoint
    loader = tf.compat.v1.train.import_meta_graph(trained_checkpoint_prefix + '.meta')
    loader.restore(sess, trained_checkpoint_prefix)

    # Export checkpoint to SavedModel
    builder = tf.compat.v1.saved_model.builder.SavedModelBuilder(export_dir)
    builder.add_meta_graph_and_variables(sess,
                                         [tf.saved_model.TRAINING, tf.saved_model.SERVING],
                                         strip_default_attrs=True)
    builder.save()

Fine-tuned BERT For Serving

In [30]:
def serving_input_fn():
    label_ids = tf.placeholder(tf.int32, [None], name='label_ids')
    input_ids = tf.placeholder(tf.int32, [None, MAX_SEQ_LENGTH], name='input_ids')
    input_mask = tf.placeholder(tf.int32, [None, MAX_SEQ_LENGTH], name='input_mask')
    segment_ids = tf.placeholder(tf.int32, [None, MAX_SEQ_LENGTH], name='segment_ids')
    input_fn = tf.estimator.export.build_raw_serving_input_receiver_fn({
        'label_ids': label_ids,
        'input_ids': input_ids,
        'input_mask': input_mask,
        'segment_ids': segment_ids,
    })()
    return input_fn

In [31]:
def predict(in_sentences):
    CLASSES = ["non-stress", "stress"]

    input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label

    input_features = run_classifier.convert_examples_to_features(input_examples, 
                                                                 label_list, 
                                                                 MAX_SEQ_LENGTH, 
                                                                 tokenizer)
    
    predict_input_fn = run_classifier.input_fn_builder(features=input_features, 
                                                       seq_length=MAX_SEQ_LENGTH, 
                                                       is_training=False, 
                                                       drop_remainder=False)

    predictions = estimator.predict(predict_input_fn)
    return [{"input": sentence, "probabilities": list(prediction['probabilities']), "class": CLASSES[prediction['labels']]}
            for sentence, prediction in zip(in_sentences, predictions)]

In [32]:
estimator._export_to_tpu = False
estimator.export_savedmodel(OUTPUT_DIR, serving_input_fn)

b'../output\\1641956961'

In [34]:
# !saved_model_cli show --dir ../output/1641956961/ --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_ids'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 310)
        name: input_ids_1:0
    inputs['input_mask'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 310)
        name: input_mask_1:0
    inputs['label_ids'] tensor_info:
        dtype: DT_INT32
        shape: (-1)
        name: label_ids_1:0
    inputs['segment_ids'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 310)
        name: segment_ids_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['labels'] tensor_info:
        dtype: DT_INT32
        shape: (-1)
        name: loss/ArgMax:0
    outputs['probabilities'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 2)
        name: loss/LogSoftmax:0
  Method name is: tensorflow/serving/predict


2022-01-12 12:10:23.265027: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_100.dll'; dlerror: cudart64_100.dll not found
2022-01-12 12:10:23.265290: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
#!tensorflow_model_server --port=8500 --rest_api_port=8501 --model_name=bert_model --model_base_path=/output

'tensorflow_model_server' is not recognized as an internal or external command,
operable program or batch file.
